<a href="https://colab.research.google.com/github/RaniaSaeed01/IE423-Tasks/blob/main/IE423_Task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
dfBank = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bank-full.csv')

In [43]:
dfBank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [44]:
# Select target as a series and features as dataframe
y = dfBank.loc[:,['y']].values.ravel()
X = dfBank.drop(['y'],axis=1)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, test_size=0.2,random_state=1)

Splitting data into numerical and categorical non-numeric data in order to do numeric analysis (Accuracy and AUC scores)

In [45]:
# Select numeric features
cols_num = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
Xnum = X[cols_num]

# Split numeric features into training and test sets
Xnum_train, Xnum_test, y_train, y_test = train_test_split(Xnum,y,train_size=0.8, test_size=0.2,random_state=1)

cols_num

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [60]:
# Select non-numeric features
cols_obj = [col for col in X.columns if X[col].dtype == 'object']
cols_obj

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome']

In [48]:
# Select categorical features
cols_cat = [col for col in X.columns if X[col].dtype == 'object' and X[col].nunique()<10]
cols_cat


['marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']

In [50]:
from sklearn.preprocessing import LabelEncoder

Xle_train = X_train.copy()
Xle_test = X_test.copy()
# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in cols_cat:
    Xle_train[col] = label_encoder.fit_transform(X_train[col])
    Xle_test[col] = label_encoder.transform(X_test[col])

In [62]:
from sklearn.preprocessing import LabelEncoder

label_encoder_y = LabelEncoder()
y_train_encoded = label_encoder_y.fit_transform(y_train)
y_test_encoded = label_encoder_y.transform(y_test)

Logistic Regression

In [63]:
# Build Logistic Regression model
from sklearn.linear_model import LogisticRegression

mdlLgrBank = LogisticRegression(random_state=1)
mdlLgrBank.fit(Xle_train[cols_num + cols_cat],y_train_encoded)
y_test_pred = mdlLgrBank.predict(Xle_test[cols_num + cols_cat])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [65]:
# Display model results using
from sklearn.metrics import confusion_matrix

print ('Confusion Matrix:')
print (confusion_matrix(y_test_encoded, y_test_pred))

Confusion Matrix:
[[7842  151]
 [ 859  191]]


In [67]:
# Display performance results
from sklearn.metrics import classification_report

print ('Classification Report (Logistic Regression):')
print (classification_report(y_test_encoded, y_test_pred))

Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      7993
           1       0.56      0.18      0.27      1050

    accuracy                           0.89      9043
   macro avg       0.73      0.58      0.61      9043
weighted avg       0.86      0.89      0.86      9043



In [68]:
# Display summary results
from sklearn.metrics import roc_auc_score

print ('AUC Score (Logistic Regression):')
print (roc_auc_score(y_test_encoded, y_test_pred))

AUC Score (Logistic Regression):
0.581506615907967


Gaussian Model

In [70]:
# Build Naive Bayes - Gaussian model
from sklearn.naive_bayes import GaussianNB

mdlNbgBank = GaussianNB()
mdlNbgBank.fit(Xle_train[cols_num + cols_cat],y_train_encoded)
y_test_pred = mdlNbgBank.predict(Xle_test[cols_num + cols_cat])

print ('Results (Naive Bayes - Gaussian):\n')

print ('Classification Report:')
print (classification_report(y_test_encoded, y_test_pred))

print ('AUC Score:')
print (roc_auc_score(y_test_encoded, y_test_pred))

Results (Naive Bayes - Gaussian):

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.89      0.91      7993
           1       0.35      0.44      0.39      1050

    accuracy                           0.84      9043
   macro avg       0.64      0.67      0.65      9043
weighted avg       0.86      0.84      0.85      9043

AUC Score:
0.6663037896254461


K-Nearest Neighbor Model

In [71]:
# Build KNN model with 5 nearest neighbors
from sklearn.neighbors import KNeighborsClassifier

mdlKnnBank = KNeighborsClassifier(n_neighbors=10)
mdlKnnBank.fit(Xle_train[cols_num + cols_cat], y_train_encoded)
y_test_pred = mdlKnnBank.predict(Xle_test[cols_num + cols_cat])

print ('AUC Score (K-Nearest Neighbor):')
print (roc_auc_score(y_test_encoded, y_test_pred))

AUC Score (K-Nearest Neighbor):
0.5871098520729447


Support Vector Machine

In [74]:
# Build Support Vector Machines - Classifier model

from sklearn.svm import SVC

mdlSvmBank = SVC()
mdlSvmBank.fit(Xle_train[cols_num + cols_cat], y_train_encoded)
y_test_pred = mdlSvmBank.predict(Xle_test[cols_num + cols_cat])

print ('Results (Support Vector Classifier):\n')

print ('Classification Report:')
print (classification_report(y_test_encoded, y_test_pred))

print ('AUC Score:')
print (roc_auc_score(y_test_encoded, y_test_pred))

Results (Support Vector Classifier):

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      7993
           1       0.41      0.01      0.01      1050

    accuracy                           0.88      9043
   macro avg       0.65      0.50      0.48      9043
weighted avg       0.83      0.88      0.83      9043

AUC Score:
0.5027077859793986


Highest Accuracy (0.89) and F-1 Score (0.94) is achieved from the logistic regression model, while the highest AUC score is from the Gaussian model.